In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import re as re
import seaborn as sns

In [95]:
# reading files
commune=pd.read_excel(r"commune.xlsx")
enroll = pd.read_csv(r"enroll.csv")
quest = pd.read_csv(r"quest.csv")
industry = pd.read_csv(r"industry.csv")
ord = pd.read_csv(r"ord.csv")
study_domain = pd.read_csv(r"study_domain.csv")
transaction = pd.read_csv(r"transaction.csv")
technology = pd.read_csv(r"technology.csv")

In [96]:
commune.head()

,Commune_en,Commune_FR,Commune_Id,Departement,ADM1_PCODE
0,Abricots,Abricots,HT0812,Grande'Anse,HT08
1,Acul du Nord,Acul du Nord,HT0321,North,HT03
2,Anse-a-Foleur,Anse-à-Foleur,HT0922,North-West,HT09
3,Anse-a-Pitre,Anse-à-Pître,HT0234,South-East,HT02
4,Anse-a-Veau,Anse-à-Veau,HT1021,Nippes,HT10


Use all the files to have a final datasets to have the following columns
   <ul>
    <li>questid</li>
    <li>gender</li>
    <li>age (age of applicant)</li>
    <li>communeName (use data prep in commune file)</li>
    <li>application_date (created at in quest file)</li>
    <li>enroll_date (created_at in enroll file)</li>
    <li>is_enroll (Yes/No) (use data prep in enroll file)</li>
    <li>Education Level</li>
   <li>Communication channels(hear_AA1)</li>
    <li>Bootcamp Insterest (after_AA)</li>
    <li>Payement Date (use ord and transaction files)</li>
    <li>Payed (Yes/No)</li>
    <li>list Technologies as columns based (use get_dummies)</li>
    <li>list  Study domains (use get_dummies)</li>
    <li>Job is formal</li>
    <li>Have computer at home</li>
    <li>Have internet at home</li>
    
   </ul>
   

## Data Processing

Show the 5 first rows on the DF

In [97]:
quest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          250 non-null    int64 
 1   gender              250 non-null    object
 2   dob                 244 non-null    object
 3   commune             250 non-null    object
 4   created_at          250 non-null    object
 5   modified_at         250 non-null    object
 6   department          249 non-null    object
 7   education_level     250 non-null    object
 8   university          250 non-null    object
 9   study_domain        250 non-null    object
 10  current_employed    250 non-null    object
 11  formal_sector_job   250 non-null    object
 12  have_computer_home  250 non-null    object
 13  internet_at_home    250 non-null    object
 14  hear_AA_1           250 non-null    object
 15  after_AA            250 non-null    object
 16  quest_id            250 no

In [98]:
enroll.head()

,Unnamed: 0,created_at,expiry_date,user_id,course_name,course_id,percentage_completed,completed_at,expired,is_free_trial,completed,started_at,activated_at,updated_at,quest_id
0,0,2020-12-12T06:41:29.844Z,NaN,29816173,Entwodiksyon Nan Syans Done,1047613,0.0,NaN,False,True,False,NaN,NaN,2020-12-12T06:41:29.844Z,NaN
1,1,2020-12-13T14:41:17.720Z,NaN,35229766,Entwodiksyon Nan Syans Done,1047613,0.0,NaN,False,True,False,NaN,NaN,2020-12-13T14:41:17.720Z,NaN
2,2,2020-12-15T00:53:06.499Z,NaN,31987560,Entwodiksyon Nan Syans Done,1047613,0.0,NaN,False,True,False,NaN,NaN,2020-12-15T00:53:06.499Z,NaN
3,3,2020-12-18T18:38:55.683Z,NaN,35616451,Entwodiksyon Nan Syans Done,1047613,0.0,NaN,False,True,False,NaN,NaN,2020-12-18T18:38:55.683Z,NaN
4,4,2020-12-21T14:45:06.528Z,NaN,35390649,Entwodiksyon Nan Syans Done,1047613,0.0,NaN,False,True,False,NaN,NaN,2020-12-21T14:45:06.528Z,NaN


In [99]:
quest["created_at"] = quest["created_at"].str[:10]

In [102]:
quest.describe()

,Unnamed: 0
count,250.000000
mean,124.500000
std,72.312977
min,0.000000
25%,62.250000
50%,124.500000
75%,186.750000
max,249.000000


In [5]:
#Format in Date 
quest["dob"].replace({"3 aout 1977": "3/08/1977"}, inplace=True)
quest["dob"] = pd.to_datetime(quest["dob"])

#Convert "created_at" in format date
quest["created_at"] = pd.to_datetime(quest["created_at"])



In [6]:
#Create a Columns "age"
quest["age"] = ((quest["created_at"]-quest["dob"])//365)
#Choose the two first character in Age
quest["age"] = quest["age"].apply(lambda x: str(x)[:2])

In [12]:
quest.head()

,Unnamed: 0,gender,dob,commune,Application_Date,modified_at,department,education_level,university,study_domain,current_employed,formal_sector_job,have_computer_home,internet_at_home,hear_AA_1,after_AA,quest_id,age
0,0,male,1998-08-25,ht0111,2021-02-14,2021-02-14T23:03:52.768462Z,NaN,unknown,unknown,[],unknown,unknown,unknown,unknown,unknown,unknown,905dc006-ce95-11eb-b2f9-7c67a234f601,22
1,1,female,1996-09-29,ht0111,2021-02-16,2021-02-16T16:01:10.861899Z,ht01,Bachelors (bacc +4),other,['other'],No,unknown,Yes,Yes,Friend,Finding a job/internship,905ddcf2-ce95-11eb-9a92-7c67a234f601,24
2,2,male,1996-05-17,ht0111,2021-02-16,2021-02-16T16:56:30.579563Z,ht01,Bachelors (bacc +4),Université d'Etat d'Haïti (UEH),[],No,unknown,Yes,Yes,Friend,Improving my data analysis skills,905e2608-ce95-11eb-809f-7c67a234f601,24
3,3,male,1996-11-24,ht0111,2021-02-16,2021-02-16T16:57:55.754582Z,ht01,Bachelors (bacc +4),Ecole Supérieure d'Infotronique d'Haïti (ESIH),"['item1', 'item6']",No,unknown,Yes,Yes,ESIH,Improving my data analysis skills,905e4cf5-ce95-11eb-96a9-7c67a234f601,24
4,4,male,1997-07-20,ht0111,2021-02-16,2021-02-16T17:05:40.907982Z,ht01,High School (16 years),unknown,['other'],No,unknown,Yes,Yes,WhatsApp,Mentorship,905e73fd-ce95-11eb-a7b0-7c67a234f601,23


In [7]:
#Rename columns Created_at in Transaction
transaction.rename(columns={"created_at":"moncash_date"}, inplace=True)

#Rename columns Created_at in Ord
ord.rename(columns={"created_at":"credit_card_date"}, inplace=True)

#Rename columns Created_at in enroll
enroll.rename(columns={"created_at":"Enroll_Date"}, inplace=True)

#rename columns created-at in quest
quest.rename(columns={"created_at":"Application_Date"}, inplace=True)

In [8]:
# list  Study domains (use get_dummies)
study_domainok1 = pd.get_dummies(data=study_domain[["key", "quest_id", "values"]], columns=['values'], prefix="", prefix_sep="")
study_domainok2 = study_domainok1.groupby("quest_id").sum()

In [9]:
#list Technologies as columns based (use get_dummies)
technologyok1 = pd.get_dummies(technology[["key", "quest_id", "values"]], columns=['values'], prefix="", prefix_sep="")
technologyok2 = technologyok1.groupby("quest_id").sum()

In [10]:
# list industry (use get_dummies)
industryok1= pd.get_dummies(industry[["key", "quest_id", "values"]], columns=['values'], prefix="", prefix_sep="")
industryok2= industryok1.groupby("quest_id").sum()

In [11]:

# Format to date
transaction.moncash_date = pd.to_datetime(transaction.moncash_date).dt.tz_localize(None)
ord.credit_card_date= pd.to_datetime(ord.credit_card_date).dt.tz_localize(None)

In [13]:
# Merge Tables by "quest_id"
querry0=pd.merge(left=quest, right=enroll, how = 'left',on='quest_id')
querry1=pd.merge(left=querry0, right=study_domainok2, how = 'left',on='quest_id')
querry2=pd.merge(left=querry1, right=technologyok2, how = 'left',on='quest_id')
querry4=pd.merge(left=querry2, right=industryok2, how = 'left',on='quest_id')


In [14]:
querry4.shape

(250, 67)

In [16]:
querry4.columns

Index(['Unnamed: 0_x', 'gender', 'dob', 'commune', 'Application_Date',
       'modified_at', 'department', 'education_level', 'university',
       'study_domain', 'current_employed', 'formal_sector_job',
       'have_computer_home', 'internet_at_home', 'hear_AA_1', 'after_AA',
       'quest_id', 'age', 'Unnamed: 0_y', 'Enroll_Date', 'expiry_date',
       'user_id', 'course_name', 'course_id', 'percentage_completed',
       'completed_at', 'expired', 'is_free_trial', 'completed', 'started_at',
       'activated_at', 'updated_at', 'Accounting', 'Computer Science',
       'Economics', 'Electrical Engineering', 'Law', 'Management', 'Medicine',
       'Statistics', 'other_x', 'Bash', 'Excel', 'Git', 'Java', 'JavaScript',
       'PHP', 'PowerBI or Tableau', 'Python', 'R', 'SQL', 'VBA', 'other_y',
       'Communications', 'Consulting', 'Education', 'Energy', 'Finance',
       'Healthcare', 'Insurance', 'Manufacturing', 'Marketing',
       'Public Sector/ Non-Profit Agencies', 'Retail/ E-Comme

In [17]:
#Merge "transaction" by "user_id"
querry5 = pd.merge(left=querry4, right=transaction, how = 'left',on='user_id')
#Merge "Ord" by "quest_id"
querry6 = pd.merge(left=querry5, right=ord, how = 'left',on='quest_id')

In [18]:
#rename columns in result6 file
querry6.rename(columns={"hear_AA_1":"communication_channel",'after_AA':"Bootcamp_Interest",'other_x':"Others_Study_Domain",'other_y':"Others_Technologies",'other':"other_industry",'Commune_FR':"commune_name"}, inplace=True)

In [19]:
querry6.head()

,Unnamed: 0_x,gender,dob,commune,Application_Date,modified_at_x,department,education_level,university,study_domain,...,user_id_y,product_name,product_id,amount_dollars,amount_cents,subscription,coupon_code,coupon_id,affiliate_referral_code,status
0,0,male,1998-08-25,ht0111,2021-02-14,2021-02-14T23:03:52.768462Z,NaN,unknown,unknown,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,female,1996-09-29,ht0111,2021-02-16,2021-02-16T16:01:10.861899Z,ht01,Bachelors (bacc +4),other,['other'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,male,1996-05-17,ht0111,2021-02-16,2021-02-16T16:56:30.579563Z,ht01,Bachelors (bacc +4),Université d'Etat d'Haïti (UEH),[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,male,1996-11-24,ht0111,2021-02-16,2021-02-16T16:57:55.754582Z,ht01,Bachelors (bacc +4),Ecole Supérieure d'Infotronique d'Haïti (ESIH),"['item1', 'item6']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,male,1997-07-20,ht0111,2021-02-16,2021-02-16T17:05:40.907982Z,ht01,High School (16 years),unknown,['other'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# create columns named "is_enroll"and  "payed"
querry6["is_enroll"] = 'No'
querry6.loc[~querry6.Enroll_Date.isna(),["is_enroll"]] ="Yes"

querry6["payed"] = 'No'
querry6.loc[~(querry6.credit_card_date.isna() & querry6.moncash_date.isna()),["payed"]] ="Yes"

In [21]:
querry6.rename(columns={"created_at": "application_date", "hear_AA_1":"channels","after_AA":"interest"}, inplace=True)

In [24]:
commune["Commune_Id"] = commune["Commune_Id"].str.lower()
commune.rename(columns = {"Commune_Id": "commune"}, inplace = True)

In [25]:
# Merge Result3 and Commune
querry7=pd.merge(left=querry6, right=commune[["Commune_FR","Departement", "commune"]], how = 'left',on='commune')

In [61]:
querry7.head()

,gender,dob,commune,Application_Date,department,education_level,university,formal_sector_job,have_computer_home,internet_at_home,...,PowerBI or Tableau,Python,R,SQL,VBA,Others_Technologies,is_enroll,payed,Commune_FR,Departement
0,male,1998-08-25,ht0111,2021-02-14,NaN,unknown,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,Yes,No,Port-au-Prince,West
1,female,1996-09-29,ht0111,2021-02-16,ht01,Bachelors (bacc +4),other,unknown,Yes,Yes,...,0.0,0.0,1.0,0.0,0.0,0.0,No,No,Port-au-Prince,West
2,male,1996-05-17,ht0111,2021-02-16,ht01,Bachelors (bacc +4),Université d'Etat d'Haïti (UEH),unknown,Yes,Yes,...,0.0,1.0,0.0,1.0,0.0,0.0,Yes,Yes,Port-au-Prince,West
3,male,1996-11-24,ht0111,2021-02-16,ht01,Bachelors (bacc +4),Ecole Supérieure d'Infotronique d'Haïti (ESIH),unknown,Yes,Yes,...,0.0,1.0,0.0,0.0,0.0,0.0,No,No,Port-au-Prince,West
4,male,1997-07-20,ht0111,2021-02-16,ht01,High School (16 years),unknown,unknown,Yes,Yes,...,0.0,0.0,0.0,0.0,0.0,0.0,No,No,Port-au-Prince,West


In [29]:
#rename the columns
querry7.rename(columns={'hear_AA_1':"Communication_Channel","after_AA":"Bootcamp_Interest","other_x":"Other_Study_Domain","created_at":"Inscription_Date"}, inplace=True)

In [31]:
querry7.columns

Index(['Unnamed: 0_x', 'gender', 'dob', 'commune', 'Application_Date',
       'modified_at_x', 'department', 'education_level', 'university',
       'study_domain', 'current_employed', 'formal_sector_job',
       'have_computer_home', 'internet_at_home', 'communication_channel',
       'Bootcamp_Interest', 'quest_id', 'age', 'Unnamed: 0_y', 'Enroll_Date',
       'expiry_date', 'user_id_x', 'course_name', 'course_id_x',
       'percentage_completed', 'completed_at', 'expired', 'is_free_trial',
       'completed', 'started_at', 'activated_at', 'updated_at', 'Accounting',
       'Computer Science', 'Economics', 'Electrical Engineering', 'Law',
       'Management', 'Medicine', 'Statistics', 'Others_Study_Domain', 'Bash',
       'Excel', 'Git', 'Java', 'JavaScript', 'PHP', 'PowerBI or Tableau',
       'Python', 'R', 'SQL', 'VBA', 'Others_Technologies', 'Communications',
       'Consulting', 'Education', 'Energy', 'Finance', 'Healthcare',
       'Insurance', 'Manufacturing', 'Marketing',
   

In [47]:
#Drop Columns
querry7.drop([ 'user_id_y',
       'product_name', 'product_id', 'amount_dollars', 'amount_cents',
       'subscription', 'coupon_code', 'coupon_id', 'affiliate_referral_code',], axis="columns" , inplace=True)

In [50]:
#Drop Columns
querry7.drop([ 'Technology (Software/ Internet)',
       'Transportation', 'other_industry', 'moncash_date', 'modified_at_y',
       'course_id_y', 'transaction_id', 'credit_card_date', 'status',], axis="columns" , inplace=True)

In [112]:
querry7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 249
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   gender                  250 non-null    object        
 1   dob                     244 non-null    datetime64[ns]
 2   commune                 250 non-null    object        
 3   Application_Date        250 non-null    datetime64[ns]
 4   education_level         250 non-null    object        
 5   university              250 non-null    object        
 6   formal_sector_job       250 non-null    object        
 7   have_computer_home      250 non-null    object        
 8   internet_at_home        250 non-null    object        
 9   communication_channel   250 non-null    object        
 10  Bootcamp_Interest       250 non-null    object        
 11  quest_id                250 non-null    object        
 12  age                     250 non-null    object    

In [109]:
querry7.columns

Index(['gender', 'dob', 'commune', 'Application_Date', 'education_level',
       'university', 'formal_sector_job', 'have_computer_home',
       'internet_at_home', 'communication_channel', 'Bootcamp_Interest',
       'quest_id', 'age', 'Accounting', 'Computer Science', 'Economics',
       'Electrical Engineering', 'Law', 'Management', 'Medicine', 'Statistics',
       'Others_Study_Domain', 'Excel', 'Git', 'Java', 'JavaScript', 'PHP',
       'PowerBI or Tableau', 'Python', 'R', 'SQL', 'Others_Technologies',
       'is_enroll', 'payed', 'Commune_FR', 'Departement'],
      dtype='object')

In [108]:
querry7.shape

(250, 36)

###  1. How many observations and variables are there in the dataset
 

In [74]:
# here your codes
x=querry7.shape
print("the dataset have "+str(x[0])+" observations and "+str(x[1])+" variables")

the dataset have 250 observations and 36 variables


### 2.A Calculate the average age of the observations
   * On the whole dataset
   * On the whole male dataset
   * On all the female dataset
### 2.B Calculate 
   * variance of the whole datatset,male and female
   * kurtosis of the whole datatset,male and female
   * skewness of the whole datatset,male and female
### 2.C Does the age distribution follow a normal distribution 
   

In [ ]:
#here your codes

### 3.Display the age frequency disbrution
   * On the whole dataset
   * On the whole male dataset
   * On all the female dataset

In [ ]:
#here your codes

### 4. Can we say that the average age (24 years old) of the observations can be considered as the average age of the population likely to participate in this bootcamp. Justify your answer

In [5]:
#here your codes

### 5. Calculate the average age of participants for each communication channel

In [6]:
#here your codes

### 6. Display an age boxplot for each communication channel

In [7]:
#here your codes

### 7 .Is there a significant age difference between these groups

In [8]:
#here your codes

### 8.Plan a BA strategy for each communication channel regarding the age group

In [9]:
#here your codes

### 9.According to the observations what is the probability of being in each channel of communication knowing your are a woman

In [10]:
#here your codes

### 10.According to the observations what is the probability of being in each channel of communication knowing your are a man

In [11]:
#here your codes

### 11. Deduce the probability of being a woman knowing each communication channel

In [12]:
#here your codes

### 13. Deduce the probability of being a woman knowing each communication channel

In [13]:
# here your codes

### 14. Deduce the probability of being a man knowing each communication channel

In [14]:

#here your codes

### 15 Display a plot to see Gender vs Communication Channels .Is there any dependency between communication channels and gender?

In [15]:
#here 

### 16 Use the same method to display plot and know if is there any dependency between communication channels and Bootcamp Insterest?

In [16]:
# here your codes

### 17.Plan a BA  strategy for each communication channel, Bootcamp Insterest regarding the gender 

In [17]:
#here your codes

### 18.Plan a BA  strategy for each communication channel, Bootcamp Insterest regarding the gender

In [18]:
# here your codes

### 19. Calculate 
    * P(Bootcamp Insterest,Communication channels,Gender/Payed=yes)
    * P(Bootcamp Insterest,Communication channels,Gender/Payed=no)

### 20 reduce 
* P(Payed="yes"/Bootcamp Insterest="Increase Skill",Communication channels="Friend",Gender="male")
* P(Payed="no"/Bootcamp Insterest="Increase Skill",Communication channels="Friend",Gender="male")

In [19]:
# here your codes

### Based on these findings, propose strategies to increase our sales?

In [20]:
#here your codes